In [ ]:
# import dontenv, cargar las variables de entorno y mostrar OPENAI_API_KEY Y TAVILY_API_KEY
import dotenv

dotenv.load_dotenv()

In [ ]:
from IPython.display import Image, display

from typing import Annotated, TypedDict, List
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# 1. Modelo y herramienta local
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
tavily_tool = TavilySearchResults(max_results=2)
tools = [tavily_tool]                       # <-- solo Tavily
llm_with_tools = llm.bind_tools(tools)

# 2. Estado
class ChatState(TypedDict):
    messages: Annotated[List, add_messages]

# 3. Nodos
def chatbot(state: ChatState):
    resp = llm_with_tools.invoke(state["messages"])
    return {"messages": [resp]}

tool_node = ToolNode(tools)

# 4. Grafo mínimal (START → chatbot → tools → chatbot …)
builder = StateGraph(ChatState)
builder.add_node("chatbot", chatbot)
builder.add_node("tools", tool_node)
builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")
graph = builder.compile(checkpointer=MemorySaver())

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # Si no tienes las dependencias de GraphViz, muestra ASCII
    print(graph.get_graph().draw_ascii())

# 5. Helper para probar
def ask(query: str):
    cfg = {"configurable": {"thread_id": "demo1"}}
    events = graph.stream({"messages": [{"role": "user", "content": query}]},
                          cfg, stream_mode="values")
    for ev in events:
        if "messages" in ev:
            print("🤖", ev["messages"][-1].content)

In [ ]:
ask("¿Quién fundó RappiPay?")

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from IPython.display import display

model = ChatOpenAI(model="gpt-4o")

async def main():
    async with MultiServerMCPClient(
        {
            "math": {
                "command": "python",
                "args": ["./math_server.py"],  # cambia la ruta si es necesario
                "transport": "stdio",
            },
            "weather": {
                "url": "http://localhost:8000/sse",
                "transport": "sse",
            }
        }
    ) as client:
        tools = client.get_tools()

        def call_model(state: MessagesState):
            response = model.bind_tools(tools).invoke(state["messages"])
            return {"messages": response}

        builder = StateGraph(MessagesState)
        builder.add_node("call_model", call_model)
        builder.add_node("tools", ToolNode(tools))

        builder.add_edge(START, "call_model")
        builder.add_conditional_edges("call_model", tools_condition)
        builder.add_edge("tools", "call_model")

        graph = builder.compile()

        try:
            display(Image(graph.get_graph().draw_mermaid_png()))
        except Exception:
            print(graph.get_graph().draw_ascii())



        math_response = await graph.ainvoke({"messages": "what's (3 + 5) x 12?"})
        print("🔢 Math:", math_response)

        weather_response = await graph.ainvoke({"messages": "what is the weather in nyc?"})
        print("🌦️ Weather:", weather_response)

# Ejecutar en notebook
await main()

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from IPython.display import display
from langgraph.graph import END

model = ChatOpenAI(model="gpt-4o")

async def main():
    async with MultiServerMCPClient(
        {
            "math": {
                "command": "python",
                "args": ["./math_server.py"],
                "transport": "stdio",
            },
            "weather": {
                "url": "http://localhost:8000/sse",
                "transport": "sse",
            }
        }
    ) as client:
        tavily_tool = TavilySearchResults(max_results=2)
        tools = client.get_tools() + [tavily_tool]
        print(tools)    

        def call_model(state: MessagesState):
            response = model.bind_tools(tools).invoke(state["messages"])
            return {"messages": response}

        builder = StateGraph(MessagesState)
        builder.add_node("call_model", call_model)
        builder.add_node("tools", ToolNode(tools))

        builder.add_edge(START, "call_model")
        builder.add_conditional_edges("call_model", tools_condition)
        builder.add_edge("tools", "call_model")

        graph = builder.compile()


        return graph  # 👈 retornar grafo compilado

# Ejecuta esto en una celda
graph = await main()

In [ ]:
agent_response = await graph.ainvoke({"messages": "what's 10 x 2 + 5?"})
for msg in agent_response["messages"]:
    print(msg.pretty_print())

In [ ]:
await graph.ainvoke({"messages": "what's the weather in Cusco?"})